<a href="https://colab.research.google.com/github/Carlos-Pessin/DNC_DataAnalisys_Python_Case1/blob/main/Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CASE**
Cliente solicitou as seguintes informações
- Número de vendas por dia
- Número de vendas e preço médio por departamento
- Idade média dos clientes de acordo com a bandeira comprada
- Qual a renda média dos clientes que compraram no último mês

**Detalhe sobre o banco de dados:**
1. Base de vendas: descrição das vendas realizadas no período de um mês
   - ID compra
   - ID canal venda
   - Bandeira
   - Data
   - Preço
   - Preço com frete
   - Nome do departamento
   - Estado
   - Cliente LOG  
2. Base de clientes: apresenta mais informações sobre os clientes
   - Cliente
   - Idade
   - Renda

- Todos os valores nulos no campo "Estado" devem ser substituídos por "MS".
- Não é aceitável que vendas tenham o campo "Preço" maior do que o campo "Preço com frete", caso haja alguma venda nesse caso, desonsiderar.

In [1]:
# import das bibliotecas
import pandas as pd
import plotly.express as px


In [2]:
# importa banco de dados do excel
vendas = pd.read_excel('/content/varejo.xlsx')

In [3]:
# visualiza BD
vendas.head()

,idcompra,idcanalvenda,bandeira,Data,Preço,Preço_com_frete,Nome_Departamento,estado,cliente_Log
0,393835,Mobile,A,2023-03-05,1582.560,791.280,Telefones e Celulares,SP,15042
1,411885,Mobile,A,2023-03-17,679.200,679.200,Tablets,SP,17590
2,437305,Mobile,A,2023-03-06,35.920,37.512,Bebes,SP,15478
3,440055,APP,A,2023-01-13,2239.416,2239.416,TVs e Acessorios,SP,2933
4,455795,Mobile,B,2023-03-31,1743.200,1785.112,Esporte e Lazer,RS,19790


In [4]:
# informacoes basicas do BD
vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80228 entries, 0 to 80227
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   idcompra           80228 non-null  int64         
 1   idcanalvenda       80228 non-null  object        
 2   bandeira           80228 non-null  object        
 3   Data               80228 non-null  datetime64[ns]
 4   Preço              79926 non-null  float64       
 5   Preço_com_frete    80228 non-null  float64       
 6   Nome_Departamento  80228 non-null  object        
 7   estado             79926 non-null  object        
 8   cliente_Log        80228 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 5.5+ MB


In [5]:
# analise geral do BD
vendas.describe()

,idcompra,Preço,Preço_com_frete,cliente_Log
count,80228.000000,79926.000000,80228.000000,80228.000000
mean,251001.882448,906.589129,908.619823,13935.464676
std,144272.920296,1059.607721,1057.583101,8065.215581
min,0.000000,6.320000,6.320000,1.000000
25%,126835.000000,144.320000,150.706000,6920.750000
50%,251760.000000,531.240000,535.120000,13938.000000
75%,375925.000000,1199.200000,1215.240000,20918.000000
max,723410.000000,14439.240000,14439.240000,27982.000000


In [6]:
# substituindo valores nulos
# por MS no estado
vendas['estado'] = vendas['estado'].fillna('MS')
# pela media no preço
media_preco = vendas['Preço'].describe()['mean']
vendas['Preço'].fillna(media_preco, inplace=True)

In [7]:
# corrigindo grafia diferente
vendas['idcanalvenda'] = vendas['idcanalvenda'].str.replace('APP', 'Aplicativo')
vendas['Nome_Departamento'] = vendas['Nome_Departamento'].str.replace(' ', '_')

In [8]:
# eliminar linha com regra - se preço > preço_com_frete
vendas = vendas.query('Preço <= Preço_com_frete')

In [9]:
# importa banco de dados do excel
cliente = pd.read_excel('cliente_varejo.xlsx')

In [10]:
# visualiza BD
cliente.head()

,cliente_Log,idade,renda
0,15042,89,11337
1,17590,54,2050
2,15478,47,8258
3,2933,39,4671
4,19790,26,9207


In [11]:
# informacoes basicas do BD
cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26425 entries, 0 to 26424
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   cliente_Log  26425 non-null  int64
 1   idade        26425 non-null  int64
 2   renda        26425 non-null  int64
dtypes: int64(3)
memory usage: 619.5 KB


In [12]:
# analise geral do BD
cliente.describe()

,cliente_Log,idade,renda
count,26425.000000,26425.000000,26425.000000
mean,13937.414456,53.412564,8241.470350
std,8066.941892,20.698825,3899.871482
min,1.000000,18.000000,1500.000000
25%,6920.000000,35.000000,4859.000000
50%,13946.000000,53.000000,8234.000000
75%,20926.000000,71.000000,11629.000000
max,27982.000000,89.000000,15000.000000


In [19]:
# renda deve ser float e nao int
cliente = cliente.astype({'renda':'float'})

In [18]:
# numero de vendas por data
vendas_por_data = vendas.groupby("Data").idcompra.nunique().reset_index()

In [20]:
px.line(vendas_por_data, x='Data', y='idcompra')

In [ ]:
# analise da quantidade de vendas por departamento
agg_dept_nvendas = vendas.groupby('Nome_Departamento').idcompra.nunique().sort_values(ascending=False).reset_index()

In [17]:
px.bar(agg_dept_nvendas, x='Nome_Departamento', y='idcompra')

In [21]:
# media de preço por departamento, ordem decrescente, arredondando pra duas casas decimais
agg_dept_preco = round(vendas.groupby('Nome_Departamento')['Preço_com_frete'].agg('mean').sort_values(ascending=False).reset_index(),2)

In [22]:
px.bar(agg_dept_preco, x='Nome_Departamento', y='Preço_com_frete')

In [23]:
# juntando as duas bases para analises
vendas_cliente = vendas.merge(cliente, how='left', on='cliente_Log')
vendas_cliente.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156003 entries, 0 to 156002
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   idcompra           156003 non-null  int64         
 1   idcanalvenda       156003 non-null  object        
 2   bandeira           156003 non-null  object        
 3   Data               156003 non-null  datetime64[ns]
 4   Preço              156003 non-null  float64       
 5   Preço_com_frete    156003 non-null  float64       
 6   Nome_Departamento  156003 non-null  object        
 7   estado             156003 non-null  object        
 8   cliente_Log        156003 non-null  int64         
 9   idade              156003 non-null  int64         
 10  renda              156003 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(4)
memory usage: 14.3+ MB


In [24]:
agg_idade_bandeira = vendas_cliente.groupby('bandeira')['idade'].agg('mean').sort_values(ascending=False).reset_index()

In [25]:
# plot com plotly
px.bar(agg_idade_bandeira, x='bandeira', y='idade')

In [42]:
# inserindo nova coluna "mes"
vendas_cliente['Mes'] = vendas['Data'].dt.month
ultmes = str(vendas_cliente['Mes'].describe()['max'])

In [60]:
vendas_ultmes = vendas_cliente.query('Mes == ' + ultmes)
vendas_ultmes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20152 entries, 7 to 80223
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   idcompra           20152 non-null  int64         
 1   idcanalvenda       20152 non-null  object        
 2   bandeira           20152 non-null  object        
 3   Data               20152 non-null  datetime64[ns]
 4   Preço              20152 non-null  float64       
 5   Preço_com_frete    20152 non-null  float64       
 6   Nome_Departamento  20152 non-null  object        
 7   estado             20152 non-null  object        
 8   cliente_Log        20152 non-null  int64         
 9   idade              20152 non-null  int64         
 10  renda              20152 non-null  float64       
 11  Mes                20152 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(4)
memory usage: 2.0+ MB


In [66]:
vendas_ultmes = vendas_cliente.query('Mes == ' + ultmes)
vendas_ultmes = vendas_ultmes.astype({'renda':'float'})
mediarenda_ultmes = vendas_ultmes['renda'].describe()['mean']

print('A renda media dos clientes que compraram no ultimo mes foi de: {:.2f}'.format(mediarenda_ultmes))

A renda media dos clientes que compraram no ultimo mes foi de: 8237.06
